# Predicción de Default de Clientes con Regresión Logística

En este cuaderno desarrolla un modelo de clasificación para predecir si un cliente
incurrirá en default el próximo mes. El dataset incluye 23 variables relacionadas
con historial crediticio, pagos y características demográficas.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
import gzip
import json
import os
import pickle
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
print("All modules imported successfully.")

All modules imported successfully.


# Paso 1.
- Realice la limpieza de los datasets:
- Renombre la columna "default payment next month" a "default".
- Remueva la columna "ID".
- Elimine los registros con informacion no disponible.
- Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, agrupe estos valores en la categoría "others".
- Renombre la columna "default payment next month" a "default"
- Remueva la columna "ID".

In [2]:
#carga de datos
train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    compression="zip",
)
test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    compression="zip",
)
print("Datos cargados correctamente.")

Datos cargados correctamente.


In [4]:
# Renombrar variable objetivo
train = train.rename(columns={"default payment next month": "default"})
test = test.rename(columns={"default payment next month": "default"})

# Eliminar columna ID 
if "ID" in train.columns:
    train = train.drop(columns=["ID"])
if "ID" in test.columns:
    test = test.drop(columns=["ID"])

# Eliminar filas con valores faltantes
train = train.dropna()
test = test.dropna()

# EDUCATION > 4 se agrupa en categoría "otros" (4)
train["EDUCATION"] = train["EDUCATION"].clip(upper=4)
test["EDUCATION"] = test["EDUCATION"].clip(upper=4)

#Eliminación de los registro de información no disponible
train = train.loc[train["EDUCATION"] != 0]
train = train.loc[train["MARRIAGE"] != 0]

test = test.loc[test["EDUCATION"] != 0]
test = test.loc[test["MARRIAGE"] != 0]

print("Datos preprocesados correctamente.")

Datos preprocesados correctamente.


# Paso 2.
- Divida los datasets en x_train, y_train, x_test, y_test.

In [8]:
x_train = train.drop(columns="default")
y_train = train["default"]

x_test = test.drop(columns="default")
y_test = test["default"]

print("Variables independientes y dependientes separadas.")

Variables independientes y dependientes separadas.


# Paso 3.
Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:

 - Transforma las variables categoricas usando el método one-hot-encoding.
 - Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
 - Estandariza la matriz de entrada.
 - Selecciona las K columnas mas relevantes de la matrix de entrada.
 - Ajusta una maquina de vectores de soporte (svm).


In [9]:
#Columnas categóricas 
categorical_features=["SEX", "EDUCATION", "MARRIAGE"]
numerical_features = num_columns = [col for col in x_train.columns if col not in categorical_features]

#Preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('scaler', StandardScaler(with_mean=True, with_std=True), numerical_features),
    ],
)

print("Preprocesador creado correctamente.")

Preprocesador creado correctamente.


# Paso 4.
- Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada. Use la función de precision
- balanceada para medir la precisión del modelo.

In [10]:
#Creamos el pipeline
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ('pca', PCA()),
        ('feature_selection', SelectKBest(score_func=f_classif)),
        ("classifier", SVC(kernel="rbf", random_state=12345, max_iter=-1)),
    ],
)
print("Pipeline creado.")
print(pipeline)

Pipeline creado.
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['SEX', 'EDUCATION',
                                                   'MARRIAGE']),
                                                 ('scaler', StandardScaler(),
                                                  ['LIMIT_BAL', 'AGE', 'PAY_0',
                                                   'PAY_2', 'PAY_3', 'PAY_4',
                                                   'PAY_5', 'PAY_6',
                                                   'BILL_AMT1', 'BILL_AMT2',
                                                   'BILL_AMT3', 'BILL_AMT4',
                                                   'BILL_AMT5', 'BILL_AMT6',
                                                   'PAY_AMT1', 'PAY_AMT2',
                                                   'PAY_AMT3', 'PAY_AMT4',
                                                   'PA

In [12]:
param_grid = {
    'pca__n_components':[20, x_train.shape[1]-2],
    'feature_selection__k':[12],
    'classifier__kernel': ['rbf'],
    'classifier__gamma': [0.1],
}

model=GridSearchCV(
    pipeline,
    param_grid,
    cv=10,
    scoring="balanced_accuracy",
    n_jobs=-1,
    refit=True
    )

model.fit(x_train, y_train)
print("Modelo entrenado correctamente.")
print("Mejores hiperparámetros encontrados:")

Modelo entrenado correctamente.
Mejores hiperparámetros encontrados:


# Paso 5.
- Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
- Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [14]:
models_dir = '../files/models'
os.makedirs(models_dir, exist_ok=True)

#Nombre del archivo
compressed_model_path = "../files/models/model.pkl.gz"

with gzip.open(compressed_model_path, "wb") as file:
    pickle.dump(model, file)
print(f"Modelo guardado en {compressed_model_path}")

Modelo guardado en ../files/models/model.pkl.gz


# Paso 6.
- Calcule las metricas de precision, precision balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba.
- Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo.
- Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. Por ejemplo:
- - {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
- - {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}
#

In [16]:
def calculate_and_save_metrics(model, X_train, X_test, y_train, y_test):
    #Prediciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Metricas para el entrenamiento
    metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'precision': precision_score(y_train, y_train_pred, zero_division=0),
        'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),
        'recall': recall_score(y_train, y_train_pred, zero_division=0),
        'f1_score': f1_score(y_train, y_train_pred, zero_division=0)
    }

    #Metricas para las pruebas
    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'precision' : precision_score(y_test, y_test_pred, zero_division=0),
        'balanced_accuracy' : balanced_accuracy_score(y_test, y_test_pred),
        'recall' : recall_score(y_test, y_test_pred, zero_division=0),
        'f1_score' : f1_score(y_test, y_test_pred, zero_division=0)
    }

    #Crear carpeta si no existe
    output_dir = '../files/output'
    os.makedirs(output_dir, exist_ok=True)

    #Guardar las metricas en un archivo JSON
    output_path = os.path.join(output_dir, 'metrics.json')
    with open(output_path, 'w') as f: #w para comenzar con un archivo limpio
        f.write(json.dumps(metrics_train) + '\n')
        f.write(json.dumps(metrics_test) + '\n')
print("Métricas calculadas y guardadas correctamente.")

Métricas calculadas y guardadas correctamente.


# Paso 7.
- Calcule las matrices de confusion para los conjuntos de entrenamiento y prueba. 
- Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. de entrenamiento o prueba. Por ejemplo:
- - {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
- -{'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#

In [18]:
def calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test):
    #Predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Calcular matrices de confusion
    cm_train = confusion_matrix(y_train, y_train_pred)
    cm_test = confusion_matrix(y_test, y_test_pred)

    #Convertir las matrices de confision en formato json
    def format_confussion_matrix(cm, dataset_type):
        return {
            'type': 'cm_matrix', 
            'dataset': dataset_type,
            'true_0': {
                'predicted_0': int(cm_train[0, 0]), 
                'predicted_1': int(cm_train[0, 1])
            },
            'true_1': {
                'predicted_0': int(cm_train[1, 0]), 
                'predicted_1': int(cm_train[1, 1])
            },
        }

    metrics = [
        format_confussion_matrix(cm_train, 'train'),
        format_confussion_matrix(cm_test, 'test')
    ]

    #Guardar las matrices de confusión en el mismo archivo json
    output_path = '../files/output/metrics.json'
    with open(output_path, 'a') as f: #'a' para agregar después de las metricas
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')

#Función principal para ejecutar todo
def main(model, X_train, X_test, y_train, y_test):
    #Crear el directorio de salida si no existe
    os.makedirs('../files/output', exist_ok=True)

    #Calcular y guardar las metricas
    calculate_and_save_metrics(model, X_train, X_test, y_train, y_test)

    #Calcular y guardar las matrices de confusión
    calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test)


print("Métricas y matrices de confusión calculadas y guardadas correctamente.")

Métricas y matrices de confusión calculadas y guardadas correctamente.
